In [45]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# read train,val and test from files

In [54]:
train = pd.read_csv('input/train.tsv', sep='\t')
test = pd.read_csv('input/test.tsv', sep='\t')

In [55]:
X = train.iloc[:,1:152]

In [56]:
y = train.loc[:,"target"]

In [ ]:
kfold = KFold(10, True, 1)

In [73]:
lasso_dict = {}
for train_index , test_index in kfold.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
    mse,zero_features = (Lasso_finetuning(X_train,y_train,X_test,y_test))
    lasso_dict[mse] = zero_features
    

alpha :  4.18082528032863e-07
alpha :  4.794795229773906e-07
graph_dim_4
graph_dim_95
graph_dim_117
alpha :  4.1576944945855885e-07
graph_dim_54
alpha :  4.1483592978719e-07
graph_dim_54
graph_dim_68
alpha :  4.811035729258782e-07
graph_dim_4
graph_dim_75
alpha :  5.924019089811373e-07
graph_dim_4
graph_dim_9
graph_dim_68
graph_dim_75
graph_dim_117
alpha :  4.450243533498008e-07
graph_dim_9
graph_dim_75
graph_dim_117
alpha :  4.46835145682801e-07
graph_dim_4
graph_dim_68
graph_dim_92
alpha :  4.459564176981794e-07
graph_dim_75
graph_dim_92
graph_dim_95
graph_dim_117
alpha :  4.149677974627478e-07
graph_dim_9


**lasso_dict** contains all pairs of "mse" : "features with zero corelation"

In [75]:
lasso_dict

{0.1114311972973531: [],
 0.11256462212376686: ['graph_dim_4', 'graph_dim_95', 'graph_dim_117'],
 0.11157049853712421: ['graph_dim_54'],
 0.11019620817951874: ['graph_dim_54', 'graph_dim_68'],
 0.10827877599015043: ['graph_dim_4', 'graph_dim_75'],
 0.11527118462141611: ['graph_dim_4',
  'graph_dim_9',
  'graph_dim_68',
  'graph_dim_75',
  'graph_dim_117'],
 0.11325168913993625: ['graph_dim_9', 'graph_dim_75', 'graph_dim_117'],
 0.11005738005577191: ['graph_dim_4', 'graph_dim_68', 'graph_dim_92'],
 0.11313180816450306: ['graph_dim_75',
  'graph_dim_92',
  'graph_dim_95',
  'graph_dim_117'],
 0.11353996158652488: ['graph_dim_9']}

**Lasso_finetuning** : returns a pair of mse of train dataset and zero corelation features

In [69]:
def Lasso_finetuning(X_train, y_train,X_test,Y_test):
    lasso = Lasso(max_iter=10000, normalize=True)
    lassocv = LassoCV(alphas=None, cv=50, max_iter=100000, normalize=True)
    lassocv.fit(X_train, y_train)
    lasso.set_params(alpha=lassocv.alpha_)
    lasso.fit(X_train, y_train)
    mse = mean_squared_error(y_test, lasso.predict(X_test))

    print("alpha : ",lassocv.alpha_)
    zero_features = []
    ser = pd.Series(lasso.coef_, index=X_train.columns)
    for i, v in ser.items():
        if (v == 0):
            print(i)
            zero_features.append(i)

    return mse,zero_features